In [ ]:
import requests
from bs4 import BeautifulSoup 
import arxiv
import urllib
import time
from tqdm.notebook import tqdm
import pickle
import sys
import os

In [49]:
soup = BeautifulSoup(open('arxiv_papers.html'), 'html.parser')
links = soup.find_all('a')
arxiv_ids = [item.get('href').split("/")[-1] for item in links]

# htmlファイルのarxiv idが若干間違ってるので修正
for i, item in enumerate(arxiv_ids):
    if len(item)<9:
        arxiv_ids[i] = item + "0"

In [65]:
# 1000個の論文を全部DL 実行注意
# searchに渡せるのは100個までっぽいので分ける
# arxivの規約で3秒に1つまでならDLするのは認められているっぽい（5秒待つ）
step = 100
for start in tqdm(range(0,1000,step)):
    search = arxiv.Search(id_list=arxiv_ids[start:start+step])
    for t, result in enumerate(tqdm(list(search.results()),leave=True)):
        for i in range(5):
            try:
                result.download_pdf(dirpath="./arxiv_papers")
                time.sleep(5) # 負荷対策
                break
            # たまに TLS/SSL connection has been closed が起こるので再試行
            except Exception as e:
                if i==4:
                    raise e
                time.sleep(10)
                print("{}: {}".fotmat(start + t,str(e)))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import tiktoken
import os

In [67]:
# 参考 https://stackoverflow.com/questions/76886954/multiple-file-loading-and-embeddings-with-openai
pdf_folder_path = "./arxiv_papers"
#print(os.listdir(pdf_folder_path))
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]

all_documents = []

with tqdm(loaders) as pbar:
    for loader in pbar:
        pbar.set_description("[Loading raw document... {}]".format(loader.file_path))
        raw_documents = loader.load()
        splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separator=". ") # Splitting text
        documents = splitter.split_documents(raw_documents)
        all_documents.extend(documents)

  0%|          | 0/1000 [00:00<?, ?it/s]

Created a chunk of size 1131, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2296, which is longer than the specified 1000
Created a chunk of size 5147, which is longer than the specified 1000
Created a chunk of size 1409, which is longer than the specified 1000
Created a chunk of size 1368, which is longer than the specified 1000
Created a chunk of size 1399, which is longer than the specified 1000
Created a chunk of size 1109, which is longer than the specified 1000
Created a chunk of size 2547, which is longer than the specified 1000
Created a chunk of size 3641, which is longer than the specified 1000
Created a chunk of size 3296, which is longer than the specified 1000
Created a chunk of size 3177, which is longer than the specified 1000
Created a chunk of size 1139, which is longer than the specified 1000
Created a chunk of size 3098, which is longer than the specified 1000
Created a chunk of s

In [13]:
encoding = tiktoken.encoding_for_model("text-embedding-ada-002")
text = ""
for doc in all_documents:
    text += doc.page_content.replace("\n", " ")
token_count = len(encoding.encode(text, allowed_special='all'))
print("予想価格: {} USD".format(token_count*0.0001/1000)) # https://openai.com/pricing > Embedding models > Model: Ada v2; Usage: $0.0001 / 1K tokens

予想価格: 2.4462737999999997 USD


In [3]:
# with open('all_documents.pkl', 'wb') as f:
#     pickle.dump(all_documents, f)
# with open('all_documents.pkl', 'rb') as f:
#     all_documents = pickle.load(f)

In [30]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
# OpenAI APIを使う処理 お金がかかる
# 一発でやろうとするとapi規制に引っかかるので何回かに分ける必要がある

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(all_documents[0:586], embeddings)
time.sleep(30)
for i in tqdm(range(1,139)):
    db2 = FAISS.from_documents(all_documents[586*i:586*(i+1)], embeddings)
    db.merge_from(db2)
    db.save_local("faiss_index_1000")
    time.sleep(30)

  0%|          | 0/138 [00:00<?, ?it/s]